In [1]:
from pathlib import Path
import os

yr = 2023
d = 10

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np

def format_input(inp):
  out = []
  for l in inp.split('\n'):
    out.append(list(l))
  return np.array(out)

In [3]:
def get_potential_directions(formatted_input, loc, first=True):
  # Looks at the current location, and all surrounding locations
  # If a surrounding location is a potential direction to leave the
  # current location AND the current location can be reached from
  # that potential location then we return that location
  # ("first" is just used to determine if we are looking at the current
  # location or one of its neighbors)


  north = (loc[0]-1, loc[1])
  south = (loc[0]+1, loc[1])
  east = (loc[0], loc[1]+1)
  west = (loc[0], loc[1]-1)

  pipe_to_potentials = {
                        '.': [],
                        '|': [north, south],
                        '-': [east, west],
                        'L': [north, east],
                        'J': [north, west],
                        '7': [south, west],
                        'F': [south, east],
                        'S': [north, south, east, west],
                        '#': [north, south, east, west]
                       }

  def is_on_map(l):
    # Is this location out of the bounds of the map
    return ((l[0]>=0 and l[0]<formatted_input.shape[0])
            and (l[1]>=0 and l[1]<formatted_input.shape[1]))

  cur_pipe = formatted_input[loc[0]][loc[1]]

  if not first:
    # We are recursively looking at a potential neighbor
    return [p for p in pipe_to_potentials[cur_pipe] if is_on_map(p)]

  potentials = []
  for p in pipe_to_potentials[cur_pipe]:
    if (is_on_map(p)
    and loc in get_potential_directions(formatted_input, p, False)):
      potentials.append(p)

  return potentials




def find_start(formatted_input):
  return tuple(x[0] for x in np.where(formatted_input=='S'))


def traverse_loop(formatted_input):

  cur_loc = find_start(formatted_input)
  cur_pipe = formatted_input[cur_loc]
  loop_locs = [cur_loc]
  while cur_pipe!='S' or len(loop_locs)==1:
    potential_dirs = get_potential_directions(formatted_input, cur_loc)
    cur_loc = [x for x in potential_dirs if x not in loop_locs
               or (x==loop_locs[0] and len(loop_locs)!=2)][0]
    cur_pipe = formatted_input[cur_loc]
    loop_locs.append(cur_loc)
  return loop_locs


def get_farthest_point_on_loop(formatted_input):
  return len(traverse_loop(formatted_input))//2



In [4]:
def print_formatted_input(formatted_input):
  print('\n'.join([''.join(''.join([x for x in y])) for y in formatted_input]))

In [5]:
def replace_non_loop_with_ground(formatted_input):
  # For anything that isn't in the loop, just
  # replace it with ground so there aren't any random pipes
  # that aren't in the loop

  loop_locs = traverse_loop(formatted_input)
  for i in range(formatted_input.shape[0]):
    for j in range(formatted_input.shape[1]):
      if (i,j) not in loop_locs:
        formatted_input[(i,j)] = '.'
  return formatted_input

In [6]:
def loc_2_expanded_loc(loc):
  return tuple([x*2 for x in loc])

def expand_input(formatted_input, loop_locs=None):
  # Basically, what we're doing here is that we're going to "expand"
  # the map so that we have a "buffer" around each location in the original map
  # this should maintain the loop, as well as denote which part of the map
  # comes from the original map, and which is in the newly added buffer
  # If given a loop_locs, then we will add pipes to complete the loop in the
  # buffer space
  expanded = np.full(tuple([2*x for x in formatted_input.shape]), '#', dtype=str)
  expanded[::2,::2] = formatted_input



  if loop_locs is not None:
    for loc in loop_locs:
      potentials = get_potential_directions(expanded,
                                            loc_2_expanded_loc(loc),
                                            False)
      cur_pipe = formatted_input[loc]
      if cur_pipe != 'S':
        for p in potentials:
          if p[0]==loc_2_expanded_loc(loc)[0]:
            expanded[p]='-'
          else:
            expanded[p]='|'


  return expanded

def collapse_input(expanded_input):
  return expanded_input[::2,::2]

In [7]:
def label_inside_locations(expanded_input, loop_locs):
  # Loop around the outside, and for each ground location, look around
  # until you either hit the edge of the map or something in the loop
  # as we do this, we will label the outside '.' locations O (outside),
  # then everything left will be inside


  to_north = lambda x: (x[0]-1, x[1])
  to_south = lambda x: (x[0]+1, x[1])
  to_east = lambda x: (x[0], x[1]+1)
  to_west = lambda x: (x[0], x[1]-1)


  l, w = expanded_input.shape
  edges = ([(0, i) for i in range(w)]
        + [(i,0) for i in range(1,l-1)]
        + [(i,w-1) for i in range(1,l-1)]
        + [(l-1,i) for i in range(w)]
          )


  # Points we've already examined
  known_outside = []
  # Points that are on our current frontier of search
  queue = []


  def is_on_map(l):
    # Is this location out of the bounds of the map
    return ((l[0]>=0 and l[0]<expanded_input.shape[0])
            and (l[1]>=0 and l[1]<expanded_input.shape[1]))

  def find_outside_neighbors(loc):
    return [p for p in [x for x in
      [to_north(loc), to_south(loc), to_east(loc), to_west(loc)]
                        if is_on_map(x)]
      if expanded_input[p] in ['.', '#']]


  # Loop around the outside edge of the map
  for loc in edges:
    if loc not in known_outside and expanded_input[loc] in ['.', '#']:
      cur_loc = loc
      queue += [x for x in find_outside_neighbors(cur_loc)
      if (x not in known_outside and x not in queue)]
      known_outside.append(cur_loc)
      while len(queue)!=0:
        cur_loc = queue.pop()
        queue += [x for x in find_outside_neighbors(cur_loc)
        if (x not in known_outside and x not in queue)]
        known_outside.append(cur_loc)


  for i in range(l):
    for j in range(w):
      if expanded_input[(i,j)] == '.':
        expanded_input[(i,j)] = 'O' if (i,j) in known_outside else 'I'


  return expanded_input

In [8]:
def count_enclosed_tiles(formatted_input, explain=False):

  loop_locs = traverse_loop(formatted_input)

  if explain:
    print("We start with a map of our input.")
    print("First, we figure out which indices are a part the loop.\n")
    print_formatted_input(formatted_input)



  replaced = replace_non_loop_with_ground(formatted_input)

  if explain:
    print("\n\n\n\nNow we will replace everything that isn't in the loop")
    print("with a ground square. (Represented by '.')\n")
    print_formatted_input(replaced)


  expanded = expand_input(replaced, loop_locs=loop_locs)

  if explain:
    print("\n\n\n\nIn order to account for 'squeezing between pipes'")
    print("we will expand the data with negative space which we will")
    print("fill in with '#'. (We also complete the loop through the")
    print("negative space)\n")
    print_formatted_input(expanded)


  labeled = label_inside_locations(expanded, loop_locs)

  if explain:
    print("\n\n\n\nNow we start an expansion from the edges of the map to")
    print("find contiguous ground and/or negative space sections that")
    print("touch the edge. We know these are all outside the loop, which we")
    print("label 'O', then the inside of the loop is labeled 'I'.")
    print("Note: We only label the ground, not the negative space\n")
    print_formatted_input(labeled)

  if explain:
    print("\n\n\n\nRemoving the negative space,")
    print("we end up with something like this.\n")
    print_formatted_input(collapse_input(labeled))
    print("\n\n\n\nNow just count the I's!")


  return np.sum(labeled == 'I')

In [9]:
toy_input = """..........
.S------7.
.|F----7|.
.||F||-||.
.||.-J.||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
.........."""


toy_formatted_input = format_input(toy_input)


count_enclosed_tiles(toy_formatted_input, explain=True)

We start with a map of our input.
First, we figure out which indices are a part the loop.

..........
.S------7.
.|F----7|.
.||F||-||.
.||.-J.||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........




Now we will replace everything that isn't in the loop
with a ground square. (Represented by '.')

..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........




In order to account for 'squeezing between pipes'
we will expand the data with negative space which we will
fill in with '#'. (We also complete the loop through the
negative space)

.#.#.#.#.#.#.#.#.#.#
####################
.#S-------------7#.#
##|#############|###
.#|#F---------7#|#.#
##|#|#########|#|###
.#|#|#.#.#.#.#|#|#.#
##|#|#########|#|###
.#|#|#.#.#.#.#|#|#.#
##|#|#########|#|###
.#|#L---7#F---J#|#.#
##|#####|#|#####|###
.#|#.#.#|#|#.#.#|#.#
##|#####|#|#####|###
.#L-----J#L-----J#.#
####################
.#.#.#.#.#.#.#.#.#.#
####################




Now we start an expansion from the edges 

4

In [12]:
import time

formatted_input = format_input(inp)

t = time.time()

print(get_farthest_point_on_loop(formatted_input))
print(count_enclosed_tiles(formatted_input))

print('\nRUNTIME: ', time.time()-t)

6903
265

RUNTIME:  42.93128824234009
